Ironhack_gambling.ipynb


In [ ]:
import pandas as pd

# Specify the file path for the Excel file
file_path = r'dataset\SQL Test Data.xlsx'

# Load the Excel file into a pandas DataFrame
try:
    # Read the Excel file
    sql_test_data = pd.ExcelFile(file_path)

    # Display all sheet names
    print("Available sheets:", sql_test_data.sheet_names)

    # Load each sheet into a dictionary of DataFrames
    ironhack_gambling_df = {sheet: sql_test_data.parse(sheet) for sheet in sql_test_data.sheet_names}

    # Display all columns in every sheet
    for sheet, df in ironhack_gambling_df.items():
        print(f"\nSheet: {sheet}")
        print(f"Columns: {df.columns.tolist()}")
        print(df.head())

    # Display the first few rows of each sheet
    for sheet, df in ironhack_gambling_df.items():
        print(f"\nSheet: {sheet}")
        print(df.head())

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found. Please check the path.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


In [ ]:
# the 'betting' sheet is problematic, I will split into 3. 

# Make a copy so we don’t accidentally mutate df
df_betting1 = df.iloc[:, 0:9].copy()   # columns 0 through 8
df_betting2 = df.iloc[:, 10:13].copy() # columns 10 through 12
df_betting3 = df.iloc[:, 14:23].copy() # columns 14 through 22




In [1]:
import pymysql
import numpy as np
import pandas as pd

# Connect to MySQL
try:
    cnx = pymysql.connect(user='root', password='Malcomx1',
                          host='localhost', database='SQLTestData')
    if cnx.open:
        print("Connection open")
    else:
        print("Connection is not successfully open")

    # The object which will interact with the database
    cursor = cnx.cursor()

    # Mapping of Excel sheet names to MySQL table names
    sheet_to_table_mapping = {
        'Account': 'account',
        'Customer': 'customer',
        'Betting': 'betting',
        'Product': 'product',
        'Student_School': ['student', 'school']
    }

    # Mapping of column names from DataFrame to MySQL schema
    table_column_mapping = {
        'account': ['account_no', 'cust_id', 'account_location', 'currency_code', 'daily_deposit_limit', 'stake_scale', 'source_prod'],
        'customer': ['cust_id', 'account_location', 'title', 'first_name', 'last_name', 'create_date', 'country_code', 'language',
                    'status', 'date_of_birth', 'contact', 'customer_group'],
        'betting': ['account_no', 'bet_date', 'class_id', 'category_id', 'source', 'bet_count', 'bet_amt', 'win_amt', 'product'],
        'product': ['class_id', 'category_id', 'product_name', 'sub_product', 'description', 'bet_or_play'],
        'student': ['student_id', 'student_name', 'city', 'school_id', 'gpa'],
        'school': ['school_id', 'school_name', 'school_city']
    }

    # Load the Excel file
    file_path = r'dataset\SQL Test Data.xlsx'
    sql_test_data = pd.ExcelFile(file_path)

    # Loop through each sheet in the mapping
    for sheet, table in sheet_to_table_mapping.items():
        # Load the Excel sheet into a DataFrame
        try:
            df = sql_test_data.parse(sheet)
        except Exception as e:
            print(f"Error loading sheet '{sheet}': {e}")
            continue

        # Clean column names
        df.columns = df.columns.str.strip()
        df.columns = df.columns.str.replace('[^a-zA-Z0-9_]', '_', regex=True)
        df.columns = df.columns.str.replace('^Unnamed.*', 'column_', regex=True)

        # Replace NaN values with None (compatible with MySQL)
        df = df.replace({np.nan: None})

        if sheet == 'Student_School':
            # Split Student_School into student and school tables
            student_start_row = 1  # Adjust based on data structure
            school_start_row = 10  # Adjust based on data structure

            # Extract and clean student data
            student_df = df.iloc[student_start_row + 1:school_start_row - 1, :5]
            student_df.columns = table_column_mapping['student']
            student_df = student_df.dropna(how='all')

            # Extract and clean school data
            school_df = df.iloc[school_start_row + 1:, [5, 6, 7]]
            school_df.columns = table_column_mapping['school']
            school_df = school_df.dropna(how='all')

            # Insert student data
            insert_query = f"INSERT IGNORE INTO student ({', '.join(table_column_mapping['student'])}) VALUES ({', '.join(['%s'] * len(table_column_mapping['student']))})"
            cursor.executemany(insert_query, [tuple(row) for _, row in student_df.iterrows()])

            # Insert school data
            insert_query = f"INSERT IGNORE INTO school ({', '.join(table_column_mapping['school'])}) VALUES ({', '.join(['%s'] * len(table_column_mapping['school']))})"
            cursor.executemany(insert_query, [tuple(row) for _, row in school_df.iterrows()])

            cnx.commit()
            print(f"Data from sheet '{sheet}' inserted into 'student' and 'school' tables successfully.")

        else:
            # Process all other sheets
            expected_columns = table_column_mapping[table]

            for col in expected_columns:
                if col not in df.columns:
                    df[col] = None  # Add missing columns with default values
            df = df[expected_columns]  # Reorder columns to match the MySQL schema

            # Create INSERT query
            insert_query = f"INSERT IGNORE INTO {table} ({', '.join(expected_columns)}) VALUES ({', '.join(['%s'] * len(expected_columns))})"

            # Insert DataFrame data into MySQL table
            try:
                cursor.executemany(insert_query, [tuple(row) for _, row in df.iterrows()])
                cnx.commit()
                print(f"Data from sheet '{sheet}' inserted into table '{table}' successfully.")
            except Exception as e:
                print(f"Error inserting data for sheet '{sheet}': {e}")

except pymysql.Error as e:
    print(f"MySQL Error: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    try:
        if 'cursor' in locals():
            cursor.close()
        if 'cnx' in locals():
            cnx.close()
        print("MySQL connection closed.")
    except Exception as e:
        print(f"Error closing connection: {e}")


Connection open
Data from sheet 'Account' inserted into table 'account' successfully.
Data from sheet 'Customer' inserted into table 'customer' successfully.
Error inserting data for sheet 'Betting': (1146, "Table 'sqltestdata.betting' doesn't exist")
Data from sheet 'Product' inserted into table 'product' successfully.
Data from sheet 'Student_School' inserted into 'student' and 'school' tables successfully.
MySQL connection closed.
